# Extract Signs from ORACC JSON
The code in this notebook will parse [ORACC](http://oracc.org) `JSON` files to extract signs from the Sumerian texts of one or more projects. 

In [1]:
import pandas as pd
import zipfile
import json
import tqdm
import requests
import errno
import os
import pickle
import re

## 0 Create Directories, if Necessary
The two directories needed for this script are `jsonzip` and `output`. If they do not exist they are created, else: do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [2]:
directories = ['jsonzip', 'output', 'corpus']
for d in directories:
    try:
        os.mkdir(d)
    except OSError as exc:
        if exc.errno !=errno.EEXIST:
            raise
        pass

## 1.1 Input Project Names
Provide a list of one or more project names, separated by commas. Note that subprojects must be listed separately, they are not included in the main project. For instance:

`epsd2/admin/ed3a, epsd2/admin/ed3b, epsd2/admin/ebla, epsd2/admin/oakk, epsd2/admin/lagash2, epsd2/admin/u3adm, epsd2/admin/u3let, epsd2/admin/u3leg, epsd2/admin/oldbab, epsd2/literary, epsd2/emesal, epsd2/earlylit, epsd2/royal, epsd2/praxis, dcclt, obmc, ckst, blms`

In [3]:
projects = input('Project(s): ').lower()

Project(s): epsd2/admin/ed3a, epsd2/admin/ed3b, epsd2/admin/ebla, epsd2/admin/oakk, epsd2/admin/lagash2, epsd2/admin/u3adm, epsd2/admin/u3let, epsd2/admin/u3leg, epsd2/admin/oldbab, epsd2/literary, epsd2/emesal, epsd2/earlylit, epsd2/royal, epsd2/praxis, dcclt, obmc, ckst, blms


## 1.2 Split the List of Projects
Split the list of projects and create a list of project names.

In [4]:
p = projects.split(',')               # split at each comma and make a list called `p`
p = [x.strip() for x in p]        # strip spaces left and right of each entry in `p`

## 1.3 Download the ZIP files
For each project in the list download all the `json` files from `http://build-oracc.museum.upenn.edu/json/`. The file is called `PROJECT.zip` (for instance: `dcclt.zip`). For subprojects the file is called `PROJECT-SUBPROJECT.zip` (for instance `cams-gkab.zip`). 

For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `zip` file may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. The `iter_content()` function in the `requests` library takes care of that.

If you have downloaded the files by hand (and put them in the `jsonzip` directory) you may skip this cell and jump directly to section [2.1 The Parsejson() function](#head21).

In [5]:
CHUNK = 16 * 1024
for project in tqdm.tqdm(p):
    project = project.replace('/', '-')
    url = "http://build-oracc.museum.upenn.edu/json/" + project + ".zip"
    file = 'jsonzip/' + project + '.zip'
    r = requests.get(url)
    if r.status_code == 200:
        print("Downloading " + url + " saving as " + file)
        with open(file, 'wb') as f:
            for c in r.iter_content(chunk_size=CHUNK):
                f.write(c)
    else:
        print(url + " does not exist.")

  0%|                                                   | 0/18 [00:00<?, ?it/s]

  6%|██▍                                        | 1/18 [00:02<00:40,  2.41s/it]

 11%|████▊                                      | 2/18 [00:08<00:56,  3.53s/it]

 17%|███████▏                                   | 3/18 [00:10<00:46,  3.07s/it]

 22%|█████████▌                                 | 4/18 [00:15<00:52,  3.75s/it]

...

 78%|████████████████████████████████▋         | 14/18 [01:27<00:16,  4.14s/it]

 83%|███████████████████████████████████       | 15/18 [01:34<00:15,  5.18s/it]

 89%|█████████████████████████████████████▎    | 16/18 [01:37<00:08,  4.45s/it]

 94%|███████████████████████████████████████▋  | 17/18 [01:39<00:03,  3.58s/it]

100%|██████████████████████████████████████████| 18/18 [01:44<00:00,  3.94s/it]


## <a name="head21"></a>2.1 The `parsejson()` function
The `parsejson()` function will "dig into" the `json` file (transformed into a dictionary) until it finds the relevant data. The `json` file consists of a hierarchy of `cdl` nodes; only the lowest nodes contain lemmatization data. The function goes down this hierarchy by calling itself when another `cdl` node is encountered. For nore information about the data hierarchy in the [ORACC](http://oracc.org) `json` files, see [ORACC Open Data](http://oracc.museum.upenn.edu/doc/opendata/index.html).

The argument of the `parsejson()` function is a `JSON` object, a dictionary that initially contains the entire contents of the original JSON file. The code takes the key `cdl` which itself contains an array (a list) of `JSON` objects. Iterating through these objects, if an object contains another `cdl` node, the function calls itself with this object as first argument. This way the function digs deeper and deeper into the `JSON` tree, until it does not encounter a `cdl` key anymore. Here we are at the level of individual words. The code checks for a key `f`, if it exists the signs are in the node `gdl` within the `f` node. 

In [6]:
def parsejson_signs(text):
    for JSONobject in text["cdl"]:
        field = ''
        if "cdl" in JSONobject: 
            parsejson_signs(JSONobject)
        if "type" in JSONobject and JSONobject["type"] == "field-start":
            field = JSONobject["subtype"]
        if "f" in JSONobject and not field in ['sg', 'pr']: # skip the fields "sign" and "pronunciation"
                                # in lexical texts
            if JSONobject["f"]["lang"][:3] == "sux": #only Sumerian and Emesal
                word = JSONobject["f"]
                f = word["form"]
                if "sexified" in word["gdl"][0]:
                    f = word["gdl"][0]["sexified"]
                if "cf" in word:
                    lemm = word["cf"] + '[' + word["gw"] + "]" + word["pos"]
                    lemm = lemm.replace(' ', '-') # remove commas and spaces from lemm
                    lemm = lemm.replace(',', '')
                else:
                    lemm = word["form"] # if word is unlemmatized
                all_.append(f)
                lemm_.append(lemm)
    return

## 2.2 Call the `parsejson()` function for every `JSON` file
The code in this cell will iterate through the list of projects entered above (1.1). For each project the `JSON` zip file is located in the directory `jsonzip`, named PROJECT.zip. 

Each of these files is extracted from the `zip` file and read with the command `json.loads()`, which reads the json data and transforms it into a Python dictionary (a sequence of keys and values).

This dictionary, which is called `text` is now sent to the `parsejson()` function. The function adds signs to the `sign_l` list.

In [7]:
all_ = []
lemm_ = []
ids_ = []
for project in p:
    file = "jsonzip/" + project.replace("/", "-") + ".zip"
    try:
        z = zipfile.ZipFile(file)       # create a Zipfile object
    except:
        print(file + " does not exist or is not a proper ZIP file")
        continue
    files = z.namelist()     # list of all the files in the ZIP
    files = [name for name in files if "corpusjson" in name and name[-5:] == '.json']                                                                                                  #that holds all the P, Q, and X numbers.
    for filename in tqdm.tqdm(files):                            #iterate over the file names
        id_no = filename[-13:-5]
        if id_no in ids_ and not "X" in id_no: # Check if P/Q number is already in there
            continue        # a text may appear in multiple projects
        id_text = project + id_no # id_text is, for instance, blms/P414332
        try:
            text = z.read(filename).decode('utf-8')         #read and decode the json file of one particular text
            data_json = json.loads(text)                # make it into a json object (essentially a dictionary)
            all_.append('Start'+id_text)
            lemm_.append('Start'+id_text)   # to keep all_ and lemm_ same length
            parsejson_signs(data_json)
            ids_.append(id_no)
            #print(filename)
        except:
            print(id_text + ' is not available or not complete')

  0%|                                                  | 0/157 [00:00<?, ?it/s]

epsd2/admin/ebla/P241764 is not available or not complete


 27%|██████████▉                             | 43/157 [00:00<00:00, 425.83it/s]

epsd2/admin/ebla/P315437 is not available or not complete


 71%|███████████████████████████▊           | 112/157 [00:00<00:00, 378.80it/s]

...

  7%|██▋                                      | 26/397 [00:00<00:09, 38.00it/s]

blms/P404904 is not available or not complete


 94%|█████████████████████████████████████▍  | 372/397 [00:09<00:00, 36.29it/s]

blms/P369092 is not available or not complete


100%|████████████████████████████████████████| 397/397 [00:09<00:00, 51.32it/s]


## 3 Data Structuring
### 3.1 Transform the Data into a DataFrame


In [8]:
words_l = []
separators = ['{', '}', '-']
separators2 = ['.', '+', '|']
operators = ['&', '%', '@', '×']
for e in tqdm.tqdm(all_):
    word = []
    if '1(šar₂{gal})' in e: # this cheating but it seems to work (appears in SKL 38)
            e = e.replace('1(šar₂{gal})', '1(šar₂)-gal')
    for s in separators: # first split word into signs   
        e = e.replace(s, ' ').strip()
    s_l = e.split()
    for sign in s_l:
        if sign[-1] == ')': # qualified sign - get only the qualifier
            stack = []  # |GIŠ×(GIŠ%GIŠ)|(LAK277) becomes LAK277
            ind = {}    # LAK277(|GIŠ×(GIŠ%GIŠ)|) becomes |GIŠ×(GIŠ%GIŠ)|
            for i, c in reversed(list(enumerate(sign))):
                if c == ')':
                    stack.append(i)
                if c == '(':
                    ind[stack.pop()] = i   # find the opening parens that belongs to the closing parens at position -1    
            start = ind[len(sign)-1]   # this line fails on 1(šar₂{gal}) in SKL.
            t = sign[start+1:-1]
            if t.isupper(): #leave 1(diš) etc. alone
                sign = t
            
        if '|' in sign:  # separate |DU.DU| and |DU+DU| into its components but not |DU&DU|
                        # and also not |DU.DU&DU|
            flag = False
            for o in operators:
                if o in sign:
                    flag = True
            if not flag:
                for s in separators2:
                    sign = sign.replace(s, ' ').strip() 
                sign_l = sign.split()
                word.extend(sign_l)
                continue
        elif "+" in sign:  # + as marker of gloss
            sign = sign.replace('+', ' ').strip()
            sign_l = sign.split()
            word.extend(sign_l)
            continue
        word.append(sign)
    words_l.append(word)           

100%|█████████████████████████████| 4337761/4337761 [01:05<00:00, 66279.45it/s]


In [9]:
with open("output/ogsl.p", "rb") as f:
    o = pickle.load(f)

In [10]:
val = list(o["value"])
utf = list(o["utf8"])
names = list(o["name"])

In [11]:
d = dict(zip(names, utf))
d2 = dict(zip(val,names))

In [12]:
names_l = []
utf8_l = []
for w in tqdm.tqdm(words_l):
    seq = [d2[s.lower()] if s.lower() in d2 else s for s in w]
    names_l.append(seq)
    utf8 = [d[n] if n in d else n for n in seq]
    utf8_l.append(''.join(utf8))

100%|████████████████████████████| 4337761/4337761 [00:40<00:00, 108241.05it/s]


In [13]:
df = pd.DataFrame({"transliteration":all_, "words":words_l, "names":names_l, "utf-8":utf8_l, "lemm" : lemm_})
df

,lemm,names,transliteration,utf-8,words
0,Startepsd2/admin/ed3a/P011046,[Startepsd2/admin/ed3a/P011046],Startepsd2/admin/ed3a/P011046,Startepsd2/admin/ed3a/P011046,[Startepsd2/admin/ed3a/P011046]
1,1(barig@c),[DIŠ],1(barig@c),𒁹,[1(barig@c)]
2,še[barley]N,[ŠE],še,𒊺,[še]
3,ba-lul,"[BA, LUL]",ba-lul,𒁀𒈜,"[ba, lul]"
4,nagar[carpenter]N,[NAGAR],nagar,𒉄,[nagar]
5,1(barig@c),[DIŠ],1(barig@c),𒁹,[1(barig@c)]
6,niŋdu[appropriate-thing]N,"[GAR, |U.GUD|]",nig₂-du₇,𒃻𒌌,"[nig₂, du₇]"
7,aŋ[measure]V/t,[|NINDA₂×NE|],ag₂,𒉘,[ag₂]
8,hur-sag-še₃-mah,"[|HI×AŠ₂|, SAG, EŠ₂, MAH]",hur-sag-še₃-mah,𒄯𒊕𒂠𒈤,"[hur, sag, še₃, mah]"
9,saŋ.DUN₃[recorder]N,"[SAG, DUN₃]",sa₁₂-du₅,𒊕𒂅,"[sa₁₂, du₅]"


In [14]:
with open("corpus/sux_df.p", "wb") as w:
    pickle.dump(df, w)

In [15]:
rep_d = {"sux_utf8": "utf-8", "sux_lemm": "lemm", "sux_tl" : "transliteration"}
for rep in rep_d:
    text = ' '.join(df[rep_d[rep]]).strip()
    text = text.replace(' Start', '\n').strip()
    text = text.replace('Start', '')
    text = re.sub(r'\n+', '\n', text)
    file = "corpus/" + rep + ".txt"
    with open(file, 'w', encoding="utf-8") as w:
        w.write(text)